# Personal HTCondor Pool Demo

A Personal HTCondor Pool is an HTCondor Pool that has a single Owner, who is defined by being:
- The pool’s only Administrator.
- The only Submitter who is allowed to submit to the pool’s SchedD.
- The Resource Owner of all resources managed by the pool.

In [1]:
import logging
from pathlib import Path

import htcondor
import classad

from personal import PersonalPool

In [2]:
logging.basicConfig(format="%(asctime)s ~ %(msg)s", level=logging.DEBUG)

We can start a personal pool by making a `PersonalPool` and calling its `start()` method.

In [ ]:
p = PersonalPool().start().detach()
p.stop()

In [3]:
pool = PersonalPool()
pool

PersonalPool(local_dir = ~/.condor/personal, state = uninitialized)

In [4]:
pool.start()
pool

2020-06-18 15:47:53,283 ~ Starting PersonalPool(local_dir = ~/.condor/personal, state = uninitialized)
2020-06-18 15:47:53,291 ~ State of PersonalPool(local_dir = ~/.condor/personal, state = initialized) changed from uninitialized to initialized
2020-06-18 15:47:53,328 ~ State of PersonalPool(local_dir = ~/.condor/personal, state = started) changed from initialized to started
2020-06-18 15:47:53,329 ~ Started condor_master (pid 2982) for PersonalPool(local_dir = ~/.condor/personal, state = started)
2020-06-18 15:47:53,346 ~ Starting up daemons for PersonalPool(local_dir = ~/.condor/personal, state = started), waiting for: COLLECTOR MASTER NEGOTIATOR SCHEDD STARTD
2020-06-18 15:47:53,355 ~ condor_who stdout was unexpectedly blank for PersonalPool(local_dir = ~/.condor/personal, state = started), retrying in 1 second (giving up in 119 seconds).
2020-06-18 15:47:54,394 ~ condor_who stdout was unexpectedly blank for PersonalPool(local_dir = ~/.condor/personal, state = started), retrying in

PersonalPool(local_dir = ~/.condor/personal, state = ready)

In [5]:
print(pool.get_config_val("LOCAL_DIR"))

/home/jovyan/.condor/personal


In [6]:
print(pool.run_command(["condor_config_val", "-dump", "LOCAL_DIR"]).stdout)

# Configuration from machine: 732f216e3e3e

# Parameters with names that match LOCAL_DIR:
FS_LOCAL_DIR = 
LOCAL_DIR = /home/jovyan/.condor/personal
# Contributing configuration file(s):
#	/home/jovyan/.condor/personal/condor_config


In [7]:
with pool.collector() as coll:
    print(coll.query()[0])


    [
        AuthenticatedIdentity = "condor@family"; 
        ResourcesUsed = 0; 
        GroupSortKey = 0.0; 
        SubtreeQuota = 0.0; 
        Name = "<none>"; 
        LastHeardFrom = 1592495279; 
        NegotiatorName = "jovyan@732f216e3e3e"; 
        DaemonStartTime = 1592495275; 
        Requested = 0.0; 
        WeightedResourcesUsed = 0.0; 
        DaemonLastReconfigTime = 1592495275; 
        UpdateSequenceNumber = 3; 
        PriorityFactor = 1.000000000000000E+03; 
        MyType = "Accounting"; 
        IsAccountingGroup = true; 
        LastUsageTime = 0; 
        SurplusPolicy = "byquota"; 
        EffectiveQuota = 0.0; 
        LastUpdate = 1592495279; 
        WeightedAccumulatedUsage = 0.0; 
        Priority = 5.000000000000000E+02; 
        AccumulatedUsage = 0.0; 
        ConfigQuota = 0.0; 
        TargetType = "none"; 
        AuthenticationMethod = "FS,PASSWORD,TOKEN"; 
        BeginUsageTime = 0; 
        AccountingGroup = "<none>"
    ]


In [8]:
sub = htcondor.Submit(
    executable = "/bin/sleep",
    arguments = "${ProcID}s",
)

with pool.schedd() as schedd:
    with schedd.transaction() as txn:
        cluster_id = sub.queue(txn, 5)

    print(f"ClusterID is {cluster_id}")
    
    for ad in schedd.query(
        constraint = f"ClusterID == {cluster_id}", 
        attr_list = ["ClusterID", "ProcID", "JobStatus"]
    ):
        print(repr(ad))

ClusterID is 1
[ ClusterID = 1; ProcID = 0; JobStatus = 1; ServerTime = 1592495281 ]
[ ClusterID = 1; ProcID = 1; JobStatus = 1; ServerTime = 1592495281 ]
[ ClusterID = 1; ProcID = 2; JobStatus = 1; ServerTime = 1592495281 ]
[ ClusterID = 1; ProcID = 3; JobStatus = 1; ServerTime = 1592495281 ]
[ ClusterID = 1; ProcID = 4; JobStatus = 1; ServerTime = 1592495281 ]


In [9]:
pool.stop()

2020-06-18 15:48:01,251 ~ Stopping PersonalPool(local_dir = ~/.condor/personal, state = ready)
2020-06-18 15:48:01,251 ~ State of PersonalPool(local_dir = ~/.condor/personal, state = stopping) changed from ready to stopping
2020-06-18 15:48:01,265 ~ condor_off succeeded for PersonalPool(local_dir = ~/.condor/personal, state = stopping): Sent "Kill-Daemon" command for "master" to local master
2020-06-18 15:48:01,265 ~ Waiting for condor_master (pid 2982) for PersonalPool(local_dir = ~/.condor/personal, state = stopping) to terminate
2020-06-18 15:48:01,280 ~ condor_master (pid 2982) for PersonalPool(local_dir = ~/.condor/personal, state = stopping) has terminated with exit code 0
2020-06-18 15:48:01,281 ~ State of PersonalPool(local_dir = ~/.condor/personal, state = stopped) changed from stopping to stopped
2020-06-18 15:48:01,281 ~ Stopped PersonalPool(local_dir = ~/.condor/personal, state = stopped)


Using the pool as a context manager makes it start/stop automatically.

In [ ]:
with PersonalPool() as pool:
    print(pool.get_config_val("LOCAL_DIR"))
    condor_status = pool.run_command(["condor_status"]).stdout
    print(condor_status)

You can't have two personal pools with the same local directory at the same time.

In [ ]:
Stoppingp1 = PersonalPool(local_dir = Path.cwd() / "target")
p2 = PersonalPool(local_dir = Path.cwd() / "target")

In [ ]:
print(p1)
print(p2)

In [ ]:
p1.start()

In [ ]:
p2.start()

In [ ]:
p1.stop()

In [ ]:
p2.start()
p2.stop()